# 1. 데이터로드

In [5]:
# 데이터 처리 및 분석
import pandas as pd
import numpy as np
import sys, os

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 문자
import re

In [3]:
import matplotlib.pyplot as plt

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

* 파일 불러오기

In [6]:
df = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_real_cleaned3.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   판례정보일련번호      5016 non-null   int64 
 1   사건번호          5016 non-null   object
 2   선고            5016 non-null   object
 3   판결유형          5016 non-null   object
 4   판시사항          3725 non-null   object
 5   판결요지          3054 non-null   object
 6   참조조문          3724 non-null   object
 7   참조판례          2450 non-null   object
 8   판례내용          5016 non-null   object
 9   판례내용_상단       5016 non-null   object
 10  판례내용_이유       5012 non-null   object
 11  판례내용_이유(전처리)  5016 non-null   object
 12  판례내용_제거(불용어)  5016 non-null   object
 13  판례내용_이유(불용어)  5012 non-null   object
dtypes: int64(1), object(13)
memory usage: 548.8+ KB


* 사본파일 생성

In [8]:
labor_df = df.copy()

In [10]:
labor_df['판례내용_이유(불용어제거)'] = labor_df['판례내용_이유(불용어)'].fillna('')
labor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   판례정보일련번호        5016 non-null   int64 
 1   사건번호            5016 non-null   object
 2   선고              5016 non-null   object
 3   판결유형            5016 non-null   object
 4   판시사항            3725 non-null   object
 5   판결요지            3054 non-null   object
 6   참조조문            3724 non-null   object
 7   참조판례            2450 non-null   object
 8   판례내용            5016 non-null   object
 9   판례내용_상단         5016 non-null   object
 10  판례내용_이유         5012 non-null   object
 11  판례내용_이유(전처리)    5016 non-null   object
 12  판례내용_제거(불용어)    5016 non-null   object
 13  판례내용_이유(불용어)    5012 non-null   object
 14  판례내용_이유(불용어제거)  5016 non-null   object
dtypes: int64(1), object(14)
memory usage: 587.9+ KB


# 2. 데이터분석

## 2-1 데이터 변환

* tf-idf

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. TF-IDF 벡터라이저 설정: 상위 10,000개의 단어만 사용하고 불용어 제거
vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)

# 2. 데이터프레임을 1000개씩 나눠서 처리할 준비
chunk_size = 1000
tfidf_result = []  # 청크 처리 결과를 저장할 리스트

print("처리 시작...")

# 3. 데이터를 청크로 나누어 처리
for i in range(0, len(labor_df), chunk_size):
    # 현재 청크 데이터 추출
    doc_chunk = labor_df['판례내용_이유(불용어제거)'][i:i+chunk_size]
    
    # TF-IDF 적용
    tfidf_array = vectorizer.fit_transform(doc_chunk)
    
    # 청크 결과를 데이터프레임으로 변환하여 리스트에 저장
    tfidf_result.append(pd.DataFrame(tfidf_array.toarray(), columns=vectorizer.get_feature_names_out()))
    
    # 청크 처리 완료 메시지 출력
    print(f"{i+chunk_size}개 문서 처리 완료")

# 5. 모든 청크를 하나의 데이터프레임으로 합침
tfidf_df = pd.concat(tfidf_result, ignore_index=True)

# 6. 최종 결과 확인 (상위 5개 출력)
print("최종 결과:")
print(tfidf_df.head())

# 7. 데이터프레임의 크기 및 처리 완료 메시지
print(f"TF-IDF 처리 완료. 최종 데이터프레임 크기: {tfidf_df.shape}")

처리 시작...
1000개 문서 처리 완료
2000개 문서 처리 완료
3000개 문서 처리 완료
4000개 문서 처리 완료
5000개 문서 처리 완료
6000개 문서 처리 완료
최종 결과:
    00  001  002  003  004  005  006  007  008  009  ...  기다릴것  김갑수  김병로  물심  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   NaN  NaN  NaN NaN   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   NaN  NaN  NaN NaN   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   NaN  NaN  NaN NaN   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   NaN  NaN  NaN NaN   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   NaN  NaN  NaN NaN   

   방준경  백한성  심안  염부  차점  파훼  
0  NaN  NaN NaN NaN NaN NaN  
1  NaN  NaN NaN NaN NaN NaN  
2  NaN  NaN NaN NaN NaN NaN  
3  NaN  NaN NaN NaN NaN NaN  
4  NaN  NaN NaN NaN NaN NaN  

[5 rows x 28664 columns]
TF-IDF 처리 완료. 최종 데이터프레임 크기: (5016, 28664)


## 2-2 클러스터링

* 클러스터링

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# TF-IDF 벡터라이저 설정
vectorizer = TfidfVectorizer(max_features=10000, min_df=2, max_df=0.95)

# TF-IDF 벡터화 수행
tfidf_array = vectorizer.fit_transform(labor_df['판례내용_이유(불용어제거)'])

# K-Means 클러스터링 설정 및 수행
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
kmeans.fit(tfidf_array)

# 클러스터 레이블을 데이터프레임에 추가
labor_df['클러스터'] = kmeans.labels_

# 결과 확인
print("클러스터링 결과:")
print(labor_df[['판례내용_이유(불용어제거)', '클러스터']].head())

클러스터링 결과:
                                      판례내용_이유(불용어제거)  클러스터
0  지난 보충 보충 한다 제척기간 도과 부당해고 최초 신청 불이익 다투는 신청 허용 근...     2
1  한다 판시와 협력 업체 고용 △△ 공장 지휘 근로 제공 근로자 파견 근로자 파견 오...     2
2  지난 보충 보충 한다 근로자 파견 판시와 사내 물류 간접 생산 공정 협력 업체 고용...     2
3  지난 서면 보충 한다 고용주 근로자 자를 수행 법률관계 파견 근로자 보호 파견 한다...     2
4  근로기준법  근로시간 사업 종류 통상 근로자 근로시간 하여 짧은 근로자 단시간 근로...     2


* 클러스터 결과 : 중심단어

In [16]:
# 6. 클러스터 중심 계산
cluster_centers = kmeans.cluster_centers_

# 7. 각 클러스터의 중심 단어 추출
def get_top_keywords_for_clusters(cluster_centers, vectorizer, n_words=10):
    order_centroids = cluster_centers.argsort()[:, ::-1]  # 각 클러스터의 중심 단어 인덱스 정렬
    terms = vectorizer.get_feature_names_out()  # 단어 목록
    
    cluster_keywords = {}
    for i in range(num_clusters):
        cluster_keywords[i] = [terms[ind] for ind in order_centroids[i, :n_words]]  # 상위 n_words 단어
    
    return cluster_keywords

# 8. 클러스터 중심 단어 출력
cluster_keywords = get_top_keywords_for_clusters(cluster_centers, vectorizer)
for cluster_num, keywords in cluster_keywords.items():
    print(f"\n클러스터 {cluster_num + 1}의 중심 단어들: {', '.join(keywords)}")


클러스터 1의 중심 단어들: 퇴직금, 퇴직, 지급, 회사, 임금, 근로자, 근로기준법, 근로, 개정, 직원

클러스터 2의 중심 단어들: 임금, 수당, 지급, 근로, 근로시간, 통상, 휴일, 휴가, 근로자, 근로기준법

클러스터 3의 중심 단어들: 근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용

클러스터 4의 중심 단어들: 소득, 세법, 과세, 금액, 법인, 징수, 부과, 세액, 소득세, 귀속

클러스터 5의 중심 단어들: 징계, 노동조합, 해고, 회사, 쟁의, 협약, 단체, 노조, 근로자, 조합원

클러스터 6의 중심 단어들: 보험, 재해, 망인, 장해, 급여, 산재, 사고, 산업, 사업, 지급


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# 클러스터별 상위 단어 및 그 단어들의 TF-IDF 값 확인
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
print("클러스터별 상위 단어 및 TF-IDF 값:")
for i in range(6):
    print(f"\n클러스터 {i+1}:")
    for ind in order_centroids[i, :10]:  # 상위 10개의 단어
        print(f"단어: {terms[ind]} - TF-IDF 값: {kmeans.cluster_centers_[i, ind]}")

클러스터별 상위 단어 및 TF-IDF 값:

클러스터 1:
단어: 퇴직금 - TF-IDF 값: 0.2999286202977807
단어: 퇴직 - TF-IDF 값: 0.13824748130998873
단어: 지급 - TF-IDF 값: 0.13266903653047707
단어: 회사 - TF-IDF 값: 0.09529023283141948
단어: 임금 - TF-IDF 값: 0.0848919291893697
단어: 근로자 - TF-IDF 값: 0.07678863983950643
단어: 근로기준법 - TF-IDF 값: 0.05021355848744424
단어: 근로 - TF-IDF 값: 0.04389408898437008
단어: 개정 - TF-IDF 값: 0.04146389599484868
단어: 직원 - TF-IDF 값: 0.04013179592059967

클러스터 2:
단어: 임금 - TF-IDF 값: 0.23828554693328
단어: 수당 - TF-IDF 값: 0.18419255432771148
단어: 지급 - TF-IDF 값: 0.1711468368434589
단어: 근로 - TF-IDF 값: 0.1006789274402318
단어: 근로시간 - TF-IDF 값: 0.09474848761839094
단어: 통상 - TF-IDF 값: 0.08641347743215562
단어: 휴일 - TF-IDF 값: 0.0858371280066576
단어: 휴가 - TF-IDF 값: 0.08417030917977646
단어: 근로자 - TF-IDF 값: 0.08044453850808203
단어: 근로기준법 - TF-IDF 값: 0.06233233890491318

클러스터 3:
단어: 근로자 - TF-IDF 값: 0.04284765952090437
단어: 회사 - TF-IDF 값: 0.039311883019359936
단어: 계약 - TF-IDF 값: 0.03676976637926475
단어: 근로 - TF-IDF 값: 0.031243462142109775
단어: 지급 

## 2-3 클러스터링 결과 주제 도출

<<<< 클러스터 주제 도출>>>>

클러스터 1 - 퇴직금과 근로조건 변경
* 퇴직금 지급, 근로기준법에 따른 퇴직 관련 규정, 근로자 권리, 법 개정 등에 대한 문제

클러스터 2 - 임금 및 근로시간
* 임금 및 수당의 지급과 근로시간, 휴일과 휴가 등의 근로기준법 관련 사항으로, 근로자의 근무 환경과 관련된 주제

클러스터 3 - 근로계약과 고용 관계
* 근로 계약의 체결과 임금 지급, 고용과 관련된 조건, 사업 내 채권·채무 등의 문제를 포함한 주제로, 회사와 근로자 간 계약과 근로 조건

클러스터 4 - 소득 및 세금 부과
* 소득세 및 법인의 소득에 대한 과세, 세금 징수와 부과, 소득의 귀속 등 세무와 관련된 문제

클러스터 5 - 노동조합과 징계
* 노동조합 및 쟁의 행위, 징계 및 해고와 관련된 문제로, 회사와 노동조합 간의 협약과 갈등

클러스터 6 - 재해와 산업재해 보험
* 재해 및 사고로 인한 장해, 산재 보험, 산업 재해 시 지급되는 급여와 보상 등에 관한 문제

# 3. 데이터 라벨링

## 3-1 전체 데이터 클러스터 분류와 라벨링

In [18]:
# 데이터별 클러스터 구분
labor_df['클러스터'] = 0

x = vectorizer.transform(labor_df['판례내용_이유(불용어제거)'])

clusters = kmeans.predict(x)

for idx, cluster in enumerate(clusters):
    print('labor_df의',(idx+1),'번째 열은 클러스터',(cluster+1),"에 속함")
    labor_df.at[idx, '클러스터'] = cluster+1

labor_df의 1 번째 열은 클러스터 3 에 속함
labor_df의 2 번째 열은 클러스터 3 에 속함
labor_df의 3 번째 열은 클러스터 3 에 속함
labor_df의 4 번째 열은 클러스터 3 에 속함
labor_df의 5 번째 열은 클러스터 3 에 속함
labor_df의 6 번째 열은 클러스터 2 에 속함
labor_df의 7 번째 열은 클러스터 3 에 속함
labor_df의 8 번째 열은 클러스터 3 에 속함
labor_df의 9 번째 열은 클러스터 3 에 속함
labor_df의 10 번째 열은 클러스터 3 에 속함
labor_df의 11 번째 열은 클러스터 3 에 속함
labor_df의 12 번째 열은 클러스터 3 에 속함
labor_df의 13 번째 열은 클러스터 3 에 속함
labor_df의 14 번째 열은 클러스터 2 에 속함
labor_df의 15 번째 열은 클러스터 5 에 속함
labor_df의 16 번째 열은 클러스터 3 에 속함
labor_df의 17 번째 열은 클러스터 1 에 속함
labor_df의 18 번째 열은 클러스터 2 에 속함
labor_df의 19 번째 열은 클러스터 3 에 속함
labor_df의 20 번째 열은 클러스터 3 에 속함
labor_df의 21 번째 열은 클러스터 3 에 속함
labor_df의 22 번째 열은 클러스터 3 에 속함
labor_df의 23 번째 열은 클러스터 3 에 속함
labor_df의 24 번째 열은 클러스터 3 에 속함
labor_df의 25 번째 열은 클러스터 3 에 속함
labor_df의 26 번째 열은 클러스터 3 에 속함
labor_df의 27 번째 열은 클러스터 3 에 속함
labor_df의 28 번째 열은 클러스터 3 에 속함
labor_df의 29 번째 열은 클러스터 6 에 속함
labor_df의 30 번째 열은 클러스터 1 에 속함
labor_df의 31 번째 열은 클러스터 3 에 속함
labor_df의 32 번째 열은 클러스터 3 에 속함
labor_df의 33 번째 열

* 클러스터 분류 및 라벨링 현황

In [19]:
labor_df['클러스터'].value_counts()

클러스터
3    2501
6     706
5     685
2     447
1     359
4     318
Name: count, dtype: int64

In [23]:
len(labor_df[labor_df['클러스터']==1])

359

* 클러스터 키워드 

In [27]:
cluster_list = {
    1: ["퇴직금", "퇴직", "지급", "회사", "임금", "근로자", "근로기준법", "근로", "개정", "직원"],
    2: ["임금", "수당", "지급", "근로", "근로시간", "통상", "휴일", "휴가", "근로자", "근로기준법"],
    3: ["근로자", "회사", "계약", "근로", "지급", "사업", "임금", "채권", "근무", "고용"],
    4: ["소득", "세법", "과세", "금액", "법인", "징수", "부과", "세액", "소득세", "귀속"],
    5: ["징계", "노동조합", "해고", "회사", "쟁의", "협약", "단체", "노조", "근로자", "조합원"],
    6: ["보험", "재해", "망인", "장해", "급여", "산재", "사고", "산업", "사업", "지급"]
}

cluster_list

{1: ['퇴직금', '퇴직', '지급', '회사', '임금', '근로자', '근로기준법', '근로', '개정', '직원'],
 2: ['임금', '수당', '지급', '근로', '근로시간', '통상', '휴일', '휴가', '근로자', '근로기준법'],
 3: ['근로자', '회사', '계약', '근로', '지급', '사업', '임금', '채권', '근무', '고용'],
 4: ['소득', '세법', '과세', '금액', '법인', '징수', '부과', '세액', '소득세', '귀속'],
 5: ['징계', '노동조합', '해고', '회사', '쟁의', '협약', '단체', '노조', '근로자', '조합원'],
 6: ['보험', '재해', '망인', '장해', '급여', '산재', '사고', '산업', '사업', '지급']}

In [28]:
labor_df['키워드'] = labor_df['클러스터'].map(cluster_list)

In [29]:
labor_df[['클러스터','키워드']].head()

,클러스터,키워드
0,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"
1,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"
2,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"
3,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"
4,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"


In [42]:
labor_df.to_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_keyword.csv', index=False)

In [44]:
labor_df.head(2)

,판례정보일련번호,사건번호,선고,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_제거_불용어,판례내용_이유_불용어제거,클러스터,키워드
0,241221,2024두32973,선고,판결,[1] 부당해고 등 구제절차에서 피신청인의 추가·변경이 허용되는지 여부(한정 적극...,[1] 부당해고 등 구제절차에서 최초 구제신청의 대상이 된 불이익처분을 다투는 범...,"[1] 근로기준법 제28조, 제29조, 노동위원회법 제23조 / [2] 근로기준법...","[1] 대법원 1996. 8. 23. 선고 95누11238 판결(공1996하, 2...","【원고, 상고인】 주식회사 ○○ (소송대리인 변호사 김소영 외 5인)【피고, 피상고...","【원고, 상고인】 주식회사 ○○ (소송대리인 변호사 김소영 외 5인)【피고, 피상고...",【이 유】 상고이유(상고이유서 제출기간이 지난 후 제출된 상고이유보충서의 기재...,{'이유': '상고이유(상고이유서 제출기간이 지난 후 제출된 상고이유보충서의 기재는...,주식회사 ○○ 김소영 피상 중앙 노동 위원장 법무 법인 지향 김진 법무 법인 서희원...,지난 보충 보충 한다 제척기간 도과 부당해고 최초 신청 불이익 다투는 신청 허용 근...,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"
1,241233,"2024다211908, 211915, 211922",선고,판결,[1] 파견근로자 보호 등에 관한 법률에 따라 직접고용의무가 발생하였으나 사용사업...,[1] 파견근로자 보호 등에 관한 법률에 따라 직접고용의무가 발생하였으나 사용사업...,"[1] 파견근로자 보호 등에 관한 법률 제6조의2 제1항, 민법 제390조, 제3...","[1] 대법원 2007. 11. 16. 선고 2005다3229 판결, 대법원 20...","【원고, 피상고인】 원고 1 외 2인 (소송대리인 변호사 조창래)【피고, 상고인】 ...","【원고, 피상고인】 원고 1 외 2인 (소송대리인 변호사 조창래)【피고, 상고인】 ...",【이 유】 상고이유를 판단한다. 1. 제1 상고이유에 대하여 원심은 판...,{'이유': '상고이유를 판단한다. 1. 제1 상고이유에 대하여 원심은 판시와 같은...,피상 조창 ○○○ 주식회사 김원정 부산 고법 창원 비용 부담 한다 한다 판시와 협력...,한다 판시와 협력 업체 고용 △△ 공장 지휘 근로 제공 근로자 파견 근로자 파견 오...,3,"[근로자, 회사, 계약, 근로, 지급, 사업, 임금, 채권, 근무, 고용]"


# 4. MySQL에 저장하기

* 열이름 괄호 삭제 후 언더바로 변경

In [ ]:
import pandas as pd

# 컬럼명 변경
labor_df.rename(columns={'판례내용_이유(전처리)': '판례내용_이유_전처리', '판례내용_제거(불용어)': '판례내용_제거_불용어', '판례내용_이유(불용어)': '판례내용_이유_불용어', '판례내용_이유(불용어제거)': '판례내용_이유_불용어제거'}, inplace=True)
labor_df = labor_df.drop('판례내용_이유_불용어')

* MySQL에 저장하기

In [48]:
labor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례정보일련번호       5016 non-null   int64 
 1   사건번호           5016 non-null   object
 2   선고             5016 non-null   object
 3   판결유형           5016 non-null   object
 4   판시사항           3725 non-null   object
 5   판결요지           3054 non-null   object
 6   참조조문           3724 non-null   object
 7   참조판례           2450 non-null   object
 8   판례내용           5016 non-null   object
 9   판례내용_상단        5016 non-null   object
 10  판례내용_이유        5012 non-null   object
 11  판례내용_이유_전처리    5016 non-null   object
 12  판례내용_제거_불용어    5016 non-null   object
 13  판례내용_이유_불용어제거  5016 non-null   object
 14  클러스터           5016 non-null   int64 
 15  키워드            5016 non-null   object
dtypes: int64(2), object(14)
memory usage: 627.1+ KB


In [ ]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

#### pandas dataframe에서는 불러올때 타입을 기본 str로 불러와서 긴 내용의 데이터 삽입을 하려면 dtype을 설정해줘야함 ####
dtype = {
    '판례내용': LONGTEXT,
    '판시사항': TEXT,
    '판결요지': TEXT,
    '참조조문': TEXT,
    '참조판례': TEXT,
    '판례내용_상단': LONGTEXT,
    '판례내용_이유': LONGTEXT,
    '판례내용_이유_전처리': LONGTEXT,
    '판례내용_제거_불용어': LONGTEXT,
    '판례내용_이유_불용어제거': LONGTEXT,
}

# CSV 파일 로드
df_labor_keyword = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_keyword.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
df_labor_keyword.to_sql('labor_keyword', con=engine, if_exists='replace', index=False, dtype=dtype)

# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")

데이터베이스 저장 완료


* 판례본문 sql 업로드 

In [ ]:
df_labor_content = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\근로_본문2.csv')

In [ ]:
df_labor_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   판례정보일련번호  5016 non-null   int64 
 1   사건번호      5016 non-null   object
 2   선고        5016 non-null   object
 3   판결유형      5016 non-null   object
 4   판시사항      3735 non-null   object
 5   판결요지      3073 non-null   object
 6   참조조문      3724 non-null   object
 7   참조판례      2450 non-null   object
 8   판례내용      5016 non-null   object
dtypes: int64(1), object(8)
memory usage: 352.8+ KB


In [ ]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

#### pandas dataframe에서는 불러올때 타입을 기본 str로 불러와서 긴 내용의 데이터 삽입을 하려면 dtype을 설정해줘야함 ####
dtype = {
    '판시사항': TEXT,
    '판결요지': TEXT,
    '참조조문': TEXT,
    '참조판례': TEXT,
    '판례내용': LONGTEXT    
}

# # CSV 파일 로드
# df_labor_keyword = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\data\df_labor_keyword.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
df_labor_content.to_sql('labor_content', con=engine, if_exists='replace', index=False, dtype=dtype)

# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")

데이터베이스 저장 완료


* null값을 가지는 판례일련번호

In [ ]:
214235
214365
213101
213631